In [13]:
# Install the trueskill library if you haven't already
# !pip install trueskill

import trueskill
import random
import pandas as pd

# Initialize the TrueSkill environment
ts = trueskill.TrueSkill(draw_probability=0.1, mu = 1000, sigma = 500)  # Adjust draw probability as needed

# Define the number of teams and matches
NUM_TEAMS = 6
NUM_MATCHES = 1

# Initialize teams with default ratings
teams = {}
for i in range(1, NUM_TEAMS + 1):
    team_name = f"Team_{i}"
    teams[team_name] = ts.Rating()
    print(f"Initialized {team_name} with rating: {teams[team_name]}")

# Function to simulate a match between two teams
def simulate_match(ts_env, team1, team2):
    # Get the current ratings
    rating1 = teams[team1]
    rating2 = teams[team2]
    
    # Calculate each team's performance by sampling from their rating distributions
    perf1 = ts_env.expose(rating1)
    perf2 = ts_env.expose(rating2)
    
    # Determine the outcome
    if perf1 > perf2:
        # Team1 wins
        ranks = [0, 1]
    elif perf1 < perf2:
        # Team2 wins
        ranks = [1.2, 0]
    else:
        # Draw
        ranks = [1, 0]

    
    # Update the ratings
    new_ratings = ts_env.rate([ [rating1], [rating2] ], ranks)
    new_ratings = ts_env.rate([ [new_ratings[0][0]], [new_ratings[1][0]] ], ranks)
    teams[team1] = new_ratings[0][0]
    teams[team2] = new_ratings[1][0]
    
    # Return the outcome
    return {
        'Team1': team1,
        'Team2': team2,
        'Team1_Rating': teams[team1],
        'Team2_Rating': teams[team2],
        'Winner': team1 if ranks[0] == 0 and ranks[1] == 1 else (team2 if ranks[0] == 1 and ranks[1] == 0 else 'Draw')
    }

# Simulate matches
match_results = []
for _ in range(NUM_MATCHES):
    # Randomly select two different teams
    team1, team2 = random.sample(list(teams.keys()), 2)
    result = simulate_match(ts, team1, team2)
    match_results.append(result)

# Create a DataFrame to display the match results
df_matches = pd.DataFrame(match_results)

# Calculate final ratings and confidence intervals
final_ratings = []
for team, rating in teams.items():
    # The rating object has a mean (mu) and standard deviation (sigma)
    # The conservative estimate of skill is called the "TrueSkill" score: mu - 3*sigma
    trueskill_score = rating.mu - 3 * rating.sigma
    final_ratings.append({
        'Team': team,
        'Rating_Mean': rating.mu,
        'Rating_StdDev': rating.sigma,
        'TrueSkill_Score': trueskill_score
    })

# Create a DataFrame to display the final team ratings
df_ratings = pd.DataFrame(final_ratings)
df_ratings.sort_values(by='TrueSkill_Score', ascending=False, inplace=True)

# Display the match results
print("Match Results:")
print(df_matches[['Team1', 'Team2', 'Winner']])

# Display the final team rankings
print("\nFinal Team Rankings:")
print(df_ratings[['Team', 'Rating_Mean', 'Rating_StdDev', 'TrueSkill_Score']])


Initialized Team_1 with rating: trueskill.Rating(mu=1000.000, sigma=500.000)
Initialized Team_2 with rating: trueskill.Rating(mu=1000.000, sigma=500.000)
Initialized Team_3 with rating: trueskill.Rating(mu=1000.000, sigma=500.000)
Initialized Team_4 with rating: trueskill.Rating(mu=1000.000, sigma=500.000)
Initialized Team_5 with rating: trueskill.Rating(mu=1000.000, sigma=500.000)
Initialized Team_6 with rating: trueskill.Rating(mu=1000.000, sigma=500.000)
Match Results:
    Team1   Team2  Winner
0  Team_1  Team_6  Team_6

Final Team Rankings:
     Team  Rating_Mean  Rating_StdDev  TrueSkill_Score
5  Team_6  1370.000049     371.474315       255.577104
0  Team_1   629.999951     371.474315      -484.422994
1  Team_2  1000.000000     500.000000      -500.000000
2  Team_3  1000.000000     500.000000      -500.000000
3  Team_4  1000.000000     500.000000      -500.000000
4  Team_5  1000.000000     500.000000      -500.000000


In [ ]:
import trueskill
import random
import pandas as pd

# Initialize the TrueSkill environment
ts = trueskill.TrueSkill(draw_probability=0.1, mu=1000, sigma=100)

# Define the number of teams and matches
NUM_TEAMS = 6
NUM_MATCHES = 100 # Increase the number of matches for clearer results

# Initialize teams with default ratings
teams = {}
for i in range(1, NUM_TEAMS + 1):
    team_name = f"Team_{i}"
    teams[team_name] = ts.Rating()
    print(f"Initialized {team_name} with rating: {teams[team_name]}")

def simulate_match(ts_env, team1, team2, goals_team1, goals_team2):
    rating1 = teams[team1]
    rating2 = teams[team2]

    # Determine the outcome for standard TrueSkill ranks
    if goals_team1 > goals_team2:
        ranks = [0, 1]
    elif goals_team1 < goals_team2:
        ranks = [1, 0]
    else:
        ranks = [0, 0]  # Let TrueSkill interpret a draw
    
    # Use margin of victory as extra updates
    print(goals_team1, goals_team2)
    goal_diff = abs(goals_team1 - goals_team2)
    print(goal_diff)
    max_goal = max(goals_team1, goals_team2)
    print(max_goal)
    print(sum([goals_team1, goals_team2]))
    print(goal_diff * max_goal / sum([goals_team1, goals_team2]))
    sum_goals = sum([goals_team1, goals_team2]) if sum([goals_team1, goals_team2]) > 0 else 1
    goal_diff = round(goal_diff * max_goal / sum_goals) # Normalize to [0, 1]
    print(f"Goal difference: {goal_diff}")

    for _ in range(1 + goal_diff):
        new_ratings = ts_env.rate([[rating1], [rating2]], ranks)
        rating1, rating2 = new_ratings[0][0], new_ratings[1][0]

    teams[team1], teams[team2] = rating1, rating2

    return {
        'Team1': team1,
        'Team2': team2,
        'Goals1': goals_team1,
        'Goals2': goals_team2,
        'Team1_Rating': rating1,
        'Team2_Rating': rating2,
        'Winner': team1 if goals_team1 > goals_team2 else (team2 if goals_team2 > goals_team1 else 'Draw')
    }

# Simulate matches with random goals
match_results = []
for _ in range(NUM_MATCHES):
    team1, team2 = random.sample(list(teams.keys()), 2)
    goals1 = random.randint(0, 5)
    goals2 = random.randint(0, 5)
    goals1 = 5
    goals2 = 0
    result = simulate_match(ts, team1, team2, goals1, goals2)
    match_results.append(result)

# Show match results
df_matches = pd.DataFrame(match_results)
print("\nMatch Results:")
print(df_matches[['Team1', 'Goals1', 'Team2', 'Goals2', 'Winner']])

# Final ratings and confidence intervals
final_ratings = []
for team, rating in teams.items():
    trueskill_score = rating.mu - 3 * rating.sigma  # Conservative skill estimate
    final_ratings.append({
        'Team': team,
        'Rating_Mean': rating.mu,
        'Rating_StdDev': rating.sigma,
        'TrueSkill_Score': trueskill_score
    })

df_ratings = pd.DataFrame(final_ratings)
df_ratings.sort_values('TrueSkill_Score', ascending=False, inplace=True)

print("\nFinal Team Rankings:")
print(df_ratings[['Team', 'Rating_Mean', 'Rating_StdDev', 'TrueSkill_Score']])

Initialized Team_1 with rating: trueskill.Rating(mu=1000.000, sigma=500.000)
Initialized Team_2 with rating: trueskill.Rating(mu=1000.000, sigma=500.000)
Initialized Team_3 with rating: trueskill.Rating(mu=1000.000, sigma=500.000)
Initialized Team_4 with rating: trueskill.Rating(mu=1000.000, sigma=500.000)
Initialized Team_5 with rating: trueskill.Rating(mu=1000.000, sigma=500.000)
Initialized Team_6 with rating: trueskill.Rating(mu=1000.000, sigma=500.000)
5 0
5
5
5
5.0
Goal difference: 5

Match Results:
    Team1  Goals1   Team2  Goals2  Winner
0  Team_1       5  Team_3       0  Team_1

Final Team Rankings:
     Team  Rating_Mean  Rating_StdDev  TrueSkill_Score
0  Team_1  1466.516875     310.222023       535.850807
2  Team_3   533.483125     310.222023      -397.182942
1  Team_2  1000.000000     500.000000      -500.000000
3  Team_4  1000.000000     500.000000      -500.000000
4  Team_5  1000.000000     500.000000      -500.000000
5  Team_6  1000.000000     500.000000      -500.00000